<a href="https://colab.research.google.com/github/basugautam/Reproducibility-Challenge-Project/blob/Architecture-Files/20Transformer_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# ✅ Cell 1: Mount Drive, Import, Load Data (Single Cell)
from google.colab import drive
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

# Load CSV file from Drive
file_path = '/content/drive/My Drive/timeseries_data.csv.csv'
df = pd.read_csv(file_path)

# Display first few rows to confirm loading
df.head()


# ✅ Cell 2: Data Exploration & Explanation
# ---------------------------------------------------------------
# a) WHY: To understand what kind of data we're dealing with
# b) HOW: We'll explore data types, summary stats, null values
# c) TERMS:
#     .info() gives column types & non-null counts
#     .describe() gives stats like mean, std, min, max
#     .isnull().sum() counts missing values in each column
# d) GOAL: Know our variables before modeling
# ---------------------------------------------------------------

# Check basic info
print("\033[94mINFO (SKY BLUE):\033[0m")
print(df.info())

# Statistical summary
print("\n\033[92mSUMMARY STATS (GREEN):\033[0m")
print(df.describe())

# Missing values
print("\n\033[91mMISSING VALUES (RED):\033[0m")
print(df.isnull().sum())


# ✅ Cell 3: Transformer-Based Model Setup (Pretrained/Custom)
# ---------------------------------------------------------------
# a) WHY: Transformer models capture long-range dependencies in sequence
# b) HOW: We'll define a basic Transformer block for time-series forecasting
# c) TERMS:
#     Encoder-Decoder: Like a translator, encoder understands input, decoder generates output
#     Self-Attention: Allows model to focus on relevant past values
#     Positional Encoding: Injects order info into sequence
# d) GOAL: Predict future values using historical time-series data
# ---------------------------------------------------------------

import torch
import torch.nn as nn
import torch.nn.functional as F

# Positional Encoding Layer
def get_positional_encoding(seq_len, d_model):
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-torch.log(torch.tensor(10000.0)) / d_model))
    pe = torch.zeros(seq_len, d_model)
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe.unsqueeze(0)

# Custom Transformer for Time Series
class TimeSeriesTransformer(nn.Module):
    def __init__(self, input_dim, d_model, nhead, num_layers, seq_length):
        super(TimeSeriesTransformer, self).__init__()
        self.seq_length = seq_length
        self.input_proj = nn.Linear(input_dim, d_model)
        self.positional_encoding = get_positional_encoding(seq_length, d_model)
        encoder_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.decoder = nn.Linear(d_model, 1)  # Predict one step ahead

    def forward(self, x):
        x = self.input_proj(x) + self.positional_encoding[:, :self.seq_length, :].to(x.device)
        x = self.transformer_encoder(x)
        x = self.decoder(x)
        return x[:, -1, :]  # return last step prediction

# Example model creation
model = TimeSeriesTransformer(input_dim=1, d_model=64, nhead=4, num_layers=2, seq_length=24)
print(model)

# Note: we would need to scale the data, create sequences of shape [batch, seq_len, 1],
# split train/test, and train this model using an optimizer + MSE loss like usual.
# This part depends on our dataset shape & how your time column is structured.



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INFO (SKY BLUE):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        55 non-null     object
 1   Unnamed: 1        28 non-null     object
 2   Unnamed: 2        28 non-null     object
 3   Unnamed: 3        55 non-null     object
 4   Unnamed: 4        55 non-null     object
 5   Unnamed: 5        55 non-null     object
 6   Unnamed: 6        55 non-null     object
 7   Unnamed: 7        55 non-null     object
 8   Unnamed: 8        55 non-null     object
 9   Unnamed: 9        55 non-null     object
 10  Dependency Ratio  55 non-null     object
 11  Unnamed: 11       55 non-null     object
 12  Unnamed: 12       55 non-null     object
 13  Median Age        55 non-null     object
 14  Unnamed: 1

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


In [7]:
# Import necessary libraries
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
import matplotlib.pyplot as plt

# Step 1: Scale the Data
scaler = MinMaxScaler(feature_range=(-1, 1))
df_scaled = scaler.fit_transform(df)  # Assuming df is the time-series data

# Step 2: Create Sequences
def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        seq = data[i:i+seq_length]
        label = data[i+seq_length, 0]  # predicting next time step value
        sequences.append(seq)
        labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = 24  # Use 24 time steps for each sequence
sequences, labels = create_sequences(df_scaled, seq_length)

# Step 3: Train/Test Split
train_size = int(len(sequences) * 0.8)  # 80% train, 20% test
X_train, X_test = sequences[:train_size], sequences[train_size:]
y_train, y_test = labels[:train_size], labels[train_size:]

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Step 4: Model Training
model = TimeSeriesTransformer(input_dim=1, d_model=64, nhead=4, num_layers=2, seq_length=seq_length)

# Set up optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

# Training loop
epochs = 20
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader)}")

# Step 5: Evaluation
model.eval()
predictions = []
true_values = []
with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        predictions.append(outputs.numpy())
        true_values.append(targets.numpy())

# Flatten the predictions and true values for plotting
predictions = np.concatenate(predictions, axis=0)
true_values = np.concatenate(true_values, axis=0)

# Step 6: Plot Results
plt.figure(figsize=(10, 6))
plt.plot(true_values, label="True Values")
plt.plot(predictions, label="Predicted Values", linestyle='dashed')
plt.legend()
plt.title("True vs Predicted Values")
plt.show()


ValueError: could not convert string to float: 'Name'